In [1]:
import django
import sys, os
from gensim.models import Word2Vec, Doc2Vec
from gensim.models import Phrases
import gensim

sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from scoping.models import *

import pandas as pd
import numpy as np

q = Query.objects.get(pk=1457)

from mongoengine import *
connect('mongoengine_documents')
import matplotlib.pyplot as plt
from mongoengine.queryset.visitor import Q
from monary import Monary
mon = Monary()

from scoping.models import *

from mongo_classes import *

In [2]:
docs = Doc.objects.filter(title__iregex='\w').order_by('id')[:5]#.filter(query=q)


In [7]:


def simple_tokenize(s):
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer('\W+', gaps=True)
    return tokenizer.tokenize(s.lower())

for d in docs:
    print(simple_tokenize(d.title))

['proof', 'of', 'concept', 'for', 'a', 'new', 'energy', 'positive', 'wastewater', 'treatment', 'scheme']
['do', 'more', 'experienced', 'developers', 'introduce', 'fewer', 'bugs']
['wettability', 'determination', 'by', 'contact', 'angle', 'measurements', 'hvbb', 'coal', 'water', 'system', 'with', 'injection', 'of', 'synthetic', 'flue', 'gas', 'and', 'co2']
['from', 'sidekick', 'to', 'sideshow', 'celebrity', 'entertainment', 'and', 'the', 'politics', 'of', 'distraction', 'why', 'americans', 'are', 'sleepwalking', 'toward', 'the', 'end', 'of', 'the', 'earth']
['gps', 'based', 'assessment', 'of', 'equity', 'impacts', 'of', 'large', 'scale', 'electronic', 'freeway', 'tolling', 'in', 'gauteng', 'south', 'africa']


In [3]:
new_model = False

if new_model:
    def read_docs(fname, tokens_only=False):
        for i, doc in enumerate(docs.iterator()):
            yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(doc.title), [doc.pk])

    titles = list(read_docs(docs))   
    model = Doc2Vec(vector_size=150, min_count=4, epochs=20)
    model.build_vocab(titles)
    %time model.train(titles, total_examples=model.corpus_count, epochs=model.epochs)
    model.save("/usr/local/apsis/queries/title_model_150")
else:
    model = Doc2Vec.load("/usr/local/apsis/queries/title_model")

In [45]:
doc = "2 degrees and Sustainable Development Goals: United theystand, divided they fall?"
#doc = "the community connection  implementation of best evidence into practice for self management of chronic diseases"
inferred_vector = model.infer_vector(
    gensim.utils.simple_preprocess(doc),
    steps=250
)
sims = model.docvecs.most_similar([inferred_vector], topn=10)

for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    doc = Doc.objects.get(pk=sims[index][0])
    print(u'%s %s: «%s»\n' % (label, sims[index], doc.title))


MOST (547314, 0.6887259483337402): «2 degrees C and SDGs: united they stand, divided they fall?»

SECOND-MOST (719548, 0.5949527025222778): «2°C and SDGs: United they stand, divided they fall?»

MEDIAN (1720151, 0.4435374140739441): «Will older sedentary people with non-insulin-dependent diabetes mellitus start exercising? A health promotion model»

LEAST (1689257, 0.42256417870521545): «COMPARATIVE SIMULATIONS OF FREE SURFACE FLOWS USING VOF-METHODS AND A NEW APPROACH FOR MULTI-SCALE INTERFACIAL STRUCTURES»



In [48]:
for index in [0,1,2,3]:
    doc = Doc.objects.get(pk=sims[index][0])
    print(f'Similarity: {sims[index][1]:.2} - {doc.title} \\\\')

Similarity: 0.69 - 2 degrees C and SDGs: united they stand, divided they fall? \\
Similarity: 0.59 - 2°C and SDGs: United they stand, divided they fall? \\
Similarity: 0.46 - Ready, steady build \\
Similarity: 0.46 - United we stand, divided we fall - Influencing international and domestic renewable energy policy \\


In [54]:
d = "Community care practices in Papua New Guinea"
dv = model.infer_vector(gensim.utils.simple_preprocess(d),steps=10)
#print(dv)
d2id = 732190
d2 = Doc.objects.get(pk=d2id).title

d2v = model.docvecs[d2id]

print(d2)

dv_sim = model.docvecs.similarity_unseen_docs(
    model,
    gensim.utils.simple_preprocess(d), 
    gensim.utils.simple_preprocess(d2),
    steps=10
)

dv_sim

print(dv_sim)

Stress in community care teams: will it affect the sustainability of community care?
0.47854826


In [52]:
import scipy
scipy.spatial.distance.cosine(dv,d2v)

0.4411402218441035

In [56]:
model.docvecs.closer_than(dv,d2v)

TypeError: unhashable type: 'numpy.ndarray'

In [11]:
doc = Doc.objects.last().title
doc = gensim.utils.simple_preprocess(doc)
#inferred_vector = model.infer_vector(gensim.utils.simple_preprocess(doc),steps=125)

#Doc2Vec.similarity_unseen_docs(model,doc,doc)
#model.docvecs.similarity_unseen_docs(doc,doc)

TypeError: similarity_unseen_docs() missing 1 required positional argument: 'doc_words2'